In [1]:
%%html
<link rel="stylesheet" href="https://code.jquery.com/ui/1.11.4/themes/smoothness/jquery-ui.css">
<link href="http://www.pitt.edu/~qim3/js/keen-dashboards.css" rel ="stylesheet" type="text/css">
<link href="https://cdnjs.cloudflare.com/ajax/libs/c3/0.4.10/c3.min.css" rel="stylesheet" type="text/css">
<link href="https://cdnjs.cloudflare.com/ajax/libs/dc/1.7.5/dc.css" rel="stylesheet" type="text/css">
<script src="http://www.pitt.edu/~qim3/js/jquery-ui-new.js"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/underscore.js/1.8.3/underscore-min.js"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.10/d3.min.js" charset="utf-8"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/crossfilter/1.3.12/crossfilter.min.js"></script>
<script src="http://www.pitt.edu/~qim3/js/test.js"> </script>
<script src= "http://www.pitt.edu/~qim3/js/c3-new.js"> </script>
<script src="http://www.pitt.edu/~qim3/js/dc-new.js"></script>
<script src="https://code.highcharts.com/highcharts.js"></script>
<script src="https://code.highcharts.com/modules/exporting.js"></script>


In [2]:
import pandas as pd
import pickle
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
import sqlite3

# Create a SQL connection to our SQLite database
con = sqlite3.connect("HGT.db")

df = pd.read_sql_query("SELECT * from hgt10", con)

#Be sure to close the connection.
con.close()

In [4]:
SH = df[df.市场 == '上海']
SH = SH.drop('id',1) # remove id column
SH = SH.drop_duplicates() # remove the duplicates
SH = SH.sort(columns = '日期')
SH_output = SH.to_json(orient='records', date_unit='s')
text_file = open("HGT_JSON.txt", "w")
text_file.write(SH_output)
text_file.close()

In [5]:
yutong = SH[SH.名称 == "宇通客车"]
yutong = yutong.sort(columns = '日期')
yutong.head()

,市场,日期,代码,名称,收盘价,涨跌幅,净买额,买入金额,卖出金额,成交金额
5417,上海,2014-11-28 00:00:00.000000,600066,宇通客车,18.67,-0.37,205870534,206029384,158850,206188234
5329,上海,2014-12-05 00:00:00.000000,600066,宇通客车,20.52,3.43,212824752,230059195,17234443,247293638
5308,上海,2014-12-08 00:00:00.000000,600066,宇通客车,21.08,2.73,125841391,164212401,38371010,202583412
5287,上海,2014-12-09 00:00:00.000000,600066,宇通客车,20.98,-0.47,233476417,242712234,9235817,251948051
5264,上海,2014-12-10 00:00:00.000000,600066,宇通客车,22.42,6.86,82719826,140461013,57741187,198202201


In [6]:
from IPython.display import Javascript
#runs arbitrary javascript, client-side
#out = Mytools.pdToJSON(MODS.data)

#out = yutong.to_json(orient='columns',date_unit='s')
out = yutong.to_json(orient='records', date_unit='s')
# There should be no code after Javascript 
Javascript("""
           window.out={};
           """.format(out))


<IPython.core.display.Javascript object>

In [7]:
%%html
<div id="container" style="min-width: 310px; height: 400px; margin: 0 auto"></div>

In [12]:
%%javascript
// out has different data order from python input

var format = d3.time.format("%Y-%m-%d");
var date = _.pluck(out,'日期');
// alert(date);
date = $.map(date,function (d){return d.slice(0,10)});
// data.forEach(function (d, index, array) {plot1.push(d.Age);});
// format.parse("2011-01-01"); // returns a Date
//alert(date);
var configure =  {
               
        chart: {
            type: 'column',
            renderTo: 'container'
        },
        title: {
            text: '宇通客车'
        },
        subtitle: {
            text: '沪港通数据'
        },
        xAxis: {
            categories: date,
            crosshair: true
        },
        yAxis: {
            //min: 0,
            title: {
                text: '金额'
            }
        },
        tooltip: {
            headerFormat: '<span style="font-size:10px">{point.key}</span><table>',
            pointFormat: '<tr><td style="color:{series.color};padding:0">{series.name}: </td>' +
                '<td style="padding:0"><b>{point.y:.1f} mm</b></td></tr>',
            footerFormat: '</table>',
            shared: true,
            useHTML: true
        },
        plotOptions: {
            column: {
                pointPadding: 0.2,
                borderWidth: 0
            }
        },
        series: [{
            name: '净买额',
            data: _.pluck(out,'净买额')

        }, {
            name: '买入金额',
            data: _.pluck(out,'买入金额')

        }, {
            name: '卖出金额',
            data: _.pluck(out,'卖出金额')

        }, {
            name: '成交金额',
            data: _.pluck(out,'成交金额')

        }]
    };

$(function () {var chart1 = new Highcharts.chart(configure)});



<IPython.core.display.Javascript object>